### Brand var

In [4]:
brand = 'cerave'
path_to_chromedriver = 'C:/Users/gianhubinh.le/Driver/chromedriver32.exe'
path_to_chromebrowser = 'C:/Users/gianhubinh.le/chrome-win64/chrome.exe'
path_to_userdata = 'C:/Users/gianhubinh.le/AppData/Local/Google/Chrome for Testing/User Data'
file_download_path = 'C:\\Users\\gianhubinh.le\\Downloads'
# file_save_path = "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Tiktok data"
file_save_path = "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/Test"

## 1. Import lib

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import xlsxwriter
import selenium
import time
import urllib3
import requests
import pandas as pd
import ipynb
from pandas import json_normalize
import json
import requests
import re
import os
import os.path
import glob
from tqdm import tqdm
import random
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta, MO
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
pd.set_option('display.max_columns', None)  

## 2. Make a date table (scrape date and timestamp to scrape)

Note for users: You must input 3 variables 
1. Start time for scraping 
2. End time for scraping 
3. Domain of scraping (1 => livestream, 2 => video, 3 => order, 4 => affiliate order) \
Example: '1,2,3' => scrape all livestream, video, order, '1' => scrape only livestream

Note for developer: \
1688212800 (1/7/2023) => all data of june \
1685620800 (1/6/2023) => all data of may

In [2]:
current_date = datetime.datetime.now()
start_date = current_date.replace(day=1).date()
end_date = (current_date - timedelta(days=1)).date()
# Format the dates as strings in "yyyy-mm-dd" format
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')
domain = '12345'

x = start_date

calendar_dict = {'date_scrape': []
                 ,'unix_time': []
                 ,'unix_time_order_start': []
                ,'unix_time_order_end': []}

df_time = pd.DataFrame(calendar_dict)
df_time['unix_time'] = df_time['unix_time'].astype(int)
df_time['unix_time_order_start'] = df_time['unix_time_order_start'].astype(int)
df_time['unix_time_order_end'] = df_time['unix_time_order_end'].astype(int)

while x <= end_date:
    
    # create a timestamp value corresponding to the date scrape (applicable to livestream + vid)
    y = x + relativedelta(days=+1)
    dt1 = datetime.datetime.combine(y, datetime.time.min)
    dt2 = dt1 + timedelta(hours=7)
    timestamp_ver1 = round(dt2.timestamp())
    
    # create a timestamp value corresponding to the date scrape (applicable to order)
    dt3 = datetime.datetime.combine(x, datetime.time.min)
    dt4 = dt3 + timedelta(hours=23) + timedelta(minutes=59)+ timedelta(seconds=59)
    timestamp_ver2 = round(dt3.timestamp())
    timestamp_ver3 = round(dt4.timestamp())
    
    # append new row to calendar
    row = {'date_scrape': x, 'unix_time': int(timestamp_ver1)
           , 'unix_time_order_start': int(timestamp_ver2), 'unix_time_order_end': int(timestamp_ver3)
          }
    row_df = pd.DataFrame(row, index = [1])
    df_time = pd.concat([df_time,row_df], ignore_index=True)

    x = x + relativedelta(days=+1) 

## 3. Renew cookie function

In [ ]:
s = Service(path_to_chromedriver)
options = webdriver.ChromeOptions()

options.binary_location = path_to_chromebrowser
options.add_argument('--user-data-dir={}'.format(path_to_userdata))
options.add_argument('--profile-directory=Profile 1')
options.add_argument('--start-maximized')
options.add_argument('--disable-infobars')

driver = webdriver.Chrome(service = s, options=options)
driver.get('https://seller-vn.tiktok.com/order?selected_sort=6&tab=all')

try:
    WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="homepage_menu_submenu_3"]/div/div[1]/span[1]/div/div[2]')))

    cookies_lst = driver.get_cookies()

    for i in cookies_lst:
        if i['name'] == 'sid_guard_tiktokseller':
            x = i['value'] 
        elif i['name'] == 'user_oec_info':  
            y = i['value'] 

    cookies_new = {
        'sid_guard_tiktokseller': x,
        'user_oec_info': y
    }

    with open("cookies_cerave.json", "w") as f:
        json.dump(cookies_new, f)
        
    print('------------------------------')
    print('Cookies is usable')
    print('------------------------------')
    
except:
    print('------------------------------')
    print('Cookies is expired, must renew')
    driver.get('https://seller-vn.tiktok.com/account/login')
    driver.find_element(By.XPATH, '//*[@id="TikTok_Ads_SSO_Login_Email_Panel_Button"]').click()
    driver.find_element(By.XPATH, '//*[@id="TikTok_Ads_SSO_Login_Email_Input"]').send_keys('123')
    driver.find_element(By.XPATH, '//*[@id="TikTok_Ads_SSO_Login_Pwd_Input"]').send_keys('123')
    driver.find_element(By.XPATH, '//*[@id="TikTok_Ads_SSO_Login_Btn"]').click()
    element = WebDriverWait(driver, 100).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="top_nav_menu_compass_v2"]')))
    
    cookies_lst = driver.get_cookies()

    for i in cookies_lst:
        if i['name'] == 'sid_guard_tiktokseller':
            x = i['value'] 
        elif i['name'] == 'user_oec_info':  
            y = i['value'] 

    cookies_new = {
        'sid_guard_tiktokseller': x,
        'user_oec_info': y
    }

    with open("cookies_cerave.json", "w") as f:
        json.dump(cookies_new, f)

    # ---------------------------------------------------------------------------------------------------#
    print('Cookies renew completed')
    print('------------------------------')

## 4. Create a function to get raw data with cookies

### 4a. Func1: get order data

4a1. Download excel file

In [26]:
start = min(df_time['unix_time_order_start'])
end = max(df_time['unix_time_order_end'])
link = f'https://seller-vn.tiktok.com/order?selected_sort=6&tab=all&time_order_created[]={start}&time_order_created[]={end}'
driver.get(link)
time.sleep(6)
# 1. Click export
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="order_export_bar"]/div/button[1]'))).click()
time.sleep(6)
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//span[normalize-space()='Export']"))).click()
time.sleep(6)
print('1. Order export completed')
# 2. Click download
while True: 
    if str(WebDriverWait(driver, 1000).until(EC.visibility_of_element_located((By.XPATH, '//div//div//div//div//div//div//div//div//div[2]//button[1]//div[1]'))).text) != 'Download':
        pass
    else: break
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, '//div//div//div//div//div//div//div//div//div[2]//button[1]//div[1]'))).click()
if len(df_time) <= 10:
    time.sleep(5)
else:
    time.sleep(10)
print('2. Order download completed')

1. Order export completed
2. Order download completed


4a2. Locate to excel file

In [27]:
# Get the list of all CSV files in the current directory
csv_files = glob.glob(file_download_path + '/*.csv')

# # Get the most recently downloaded CSV file
most_recent_file = max(csv_files, key=os.path.getctime)

# # Print the name of the most recently downloaded CSV file
df_order = pd.read_csv(most_recent_file)

### 4b. Func2: get order affiliate data

4b1. Download excel file

In [28]:
# 1. Request a download file
driver.get('https://affiliate.tiktok.com/product/order?shop_region=VN')

with open("cookies_cerave.json", "r") as f:
        cookies = json.load(f)

json_data = {
    'conditions': {
        'time_period': {
            'beginning_time': str(min(df_time['unix_time_order_start'])) + '000',
            'ending_time': str(max(df_time['unix_time_order_end'])) + '000',
        },
        'filter_type': 0,
        'file_format': 0,
        'product_id': '',
    },
}

response = requests.post(
    'https://affiliate.tiktok.com/api/v1/affiliate/export_order_v2'
    cookies=cookies,
    json=json_data,
    verify=False
)

if response.text.find("success") != -1: 
    print('1. Order affiliate export completed')
else:     
    print('1. Order affiliate export failed')

time.sleep(5)    
    
# 2. Locate to download location and download file
# click exported
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//span[normalize-space()='Exported']"))).click()
# click download: only download if file is: ready to download and is a new file
# ready to download
while True: 
    ready_to_download = WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/span/div[1]/div/div/div/div/div/div[3]/div[1]/div[2]/button')))
    if ready_to_download.get_attribute('disabled') == 'true':
        pass
    else: break
# new file
while True:
    try:
        driver.find_element(By.XPATH, '/html/body/div[2]/span/div[1]/div/div/div/div/div/div[3]/div[1]/div[2]/span')
        break
    except:    
        pass
    
time.sleep(2)    
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[2]/span/div[1]/div/div/div/div/div/div[3]/div[1]/div[2]/button'))).click()
if len(df_time) <= 10:
    time.sleep(10)
else:
    time.sleep(20)
print('2. Order affiliate download completed')

1. Order affiliate export completed
2. Order affiliate download completed


4b2. Locate to excel file

In [29]:
# Get the list of all CSV files in the current directory
csv_files = glob.glob(file_download_path + '/*.csv')

# # Get the most recently downloaded CSV file
most_recent_file = max(csv_files, key=os.path.getctime)

# # Print the name of the most recently downloaded CSV file
df_order_affiliate = pd.read_csv(most_recent_file, dtype={
    'Content ID': 'string'
    ,'Order ID': 'string'
    ,'Product ID': 'string'
    ,'Sku Id': 'string'
})

driver.quit()

### 4c. Func3: get live stream data

In [8]:
def get_data_livestream(x):
    
    with open("cookies_cerave.json", "r") as f:
        cookies = json.load(f)

    json_data = {
        'request': {
            'params': [
                {
                    'list_control': {
                        'pagination': {
                            'size': 1000,
                            'page': 0,
                        },
                        'rules': [
                            {
                                'direction': 2,
                                'field': 'LIVE_LIST_START_TIMESTAMP',
                            },
                        ],
                    },
                    'filter': {
                        'creator_id': [],
                    },
                    'time_selector': {
                        'period': 20,
                        'granularity': 1,
                        'base_timestamp': int(x),
                        'timezone_offset': 0,
                    },
                    'stats_types': [
                        12,20,61,35,36,30,37,21,31,34,86,32,81,87,41,83,80,85,33,40,84,82,2,3,4,5,6,10,12,13,14,7,8,
                    ],
                },
            ],
        },
    }
    
    while True:
        try:
            response = requests.post(
                'https://seller-vn.tiktok.com/api/v2/insights/seller/live/list,
                cookies=cookies,
                headers=headers,
                json=json_data,
                verify=False)
            
            data = response.json()
    
            global df_livestream

            if 'stats' in data['data']['segments'][0]['timed_lists'][0]:
                lst = data['data']['segments'][0]['timed_lists'][0]['stats']
                df_livestream = pd.json_normalize(lst, max_level=0)
            else:
                df_livestream = pd.DataFrame(columns=['A'])
        
            break
        except:
            print("Connection refused by the server..")
            print("Let me sleep for 3 seconds")
            print("ZZzzzz...")
            time.sleep(3)
            print("Was a nice sleep, now let me continue...")
            continue

def revenue_clean(x):
    return x['amount']

def avg_unit_price_clean(x):
    return x['amount']

### 4d. Func4: get video data

In [9]:
def get_data_video(x):
    
    with open("cookies_cerave.json", "r") as f:
        cookies = json.load(f)

    json_data = {
        'request': {
            'params': [
                {
                    'time_selector': {
                        'period': 20,
                        'granularity': 11,
                        'base_timestamp': int(x),
                        'timezone_offset': 0,
                    },
                    'list_control': {
                        'rules': [
                            {
                                'direction': 2,
                                'field': 'SHOP_VIDEO_LIST_REVENUE',
                            },
                        ],
                        'pagination': {
                            'size': 1000,
                            'page': 0,
                        },
                    },
                    'stats_types': [
                        2,4,3,80,90,62,82,63,85,120,81,20,60,200,22,61,21,23,24,
                    ],
                },
            ],
        },
    }

  
    while True:
        try:
            response = requests.post(
                'https://seller-vn.tiktok.com/api/v2/insights/seller/shop/video/list'
                cookies=cookies,
                headers=headers,
                json=json_data,
                verify=False)
            
            data = response.json()
    
            global df_video

            if 'stats' in data['data']['segments'][0]['timed_lists'][0]:
                lst = data['data']['segments'][0]['timed_lists'][0]['stats']
                df_video = pd.json_normalize(lst, max_level=0)
            else:
                df_video = pd.DataFrame(columns=['A'])
        
            break
        except:
            print("Connection refused by the server..")
            print("Let me sleep for 3 seconds")
            print("ZZzzzz...")
            time.sleep(3)
            print("Was a nice sleep, now let me continue...")
            continue    

# 1. Clean creator        
def creator_id(x): 
    if type(x) == float:
        return None
    else: return x['id']

def creator_handle(x): 
    if type(x) == float:
        return None
    else: return x['handle']

def creator_alias(x): 
    if type(x) == float:
        return None
    else: return x['alias']

# 2. Clean video  
def video_id(x): return x['id']

def video_name(x): 
    match = re.search(r"'name':\s+'([^']*)'", x)
    match2 = re.search(r"'name': \"(.*?)\"", x)
    if match:
        name_value = match.group(1)
        return name_value
    elif match2:
        name_value = match2.group(1)
        return name_value

def video_url(x): 
    match = re.search(r"'main_url': '(.*?)'", x)
    if match:
        name_value = match.group(1)
        return name_value
    
def video_publish_time(x): 
    match = re.search(r"'publish_time':\s+(\d+)", x) 
    if match:
        name_value = match.group(1)
        return name_value   

def video_duration(x): 
    match = re.search(r"'duration':\s+([0-9.]+)", x)
    if match:
        duration_value = match.group(1)
        return duration_value

# 3. Clean number
def video_revenue(x): return x['amount']
def video_refunds(x): return x['amount']
def video_commission_estimated(x): return x['amount']

### 4e. Func5: get product data

In [10]:
def get_data_product(x):
    with open("cookies_cerave.json", "r") as f:
            cookies = json.load(f)

    params = {
        'locale': 'en',
        'language': 'en',
        'oec_seller_id': '7494854840010246582',
        'aid': '4068',
        'app_name': 'i18n_ecom_shop',
    }
    
    json_data = {
        'request': {
            'params': [
                {
                    'stats_types': [
                        10,21,20,23,11,30,37,120,1,33,38,32,34,121,12,40,47,220,41,43,48,42,44,221,2,4,3,270,
                    ],
                    'time_selector': {
                        'period': 20,
                        'granularity': 1,
                        'base_timestamp': int(x),
                        'timezone_offset': 0,
                    },
                    'search_filter': {
                        'input': '',
                    },
                    'list_control': {
                        'rules': [
                            {
                                'direction': 2,
                                'field': 'SHOP_PRODUCT_LIST_REVENUE',
                            },
                        ],
                        'pagination': {
                            'size': 1000,
                            'page': 0,
                        },
                    },
                },
            ],
        },
    }
    
    response = requests.post(
        'https://seller-vn.tiktok.com/api/v2/insights/seller/shop/product/list',
        params=params,
        cookies=cookies,
        json=json_data,
        verify=False
    )
    data = response.json()
    
    while True:
        try:
            if 'stats' in data['data']['segments'][0]['timed_lists'][0]:
                lst = data['data']['segments'][0]['timed_lists'][0]['stats']
                global df_product
                df_product = pd.json_normalize(lst, max_level=0)
                break
            else:
                columns = ['shop_id', 'product_id', 'product_name', 'product_image','revenue', 'revenue_live', 'revenue_video',
                    'buyer_cnt', 'item_sold_cnt', 'sku_order_cnt','item_sold_live_cnt', 'buyer_live_cnt', 'viewer_live_cnt',
                    'unique_item_click_live_cnt', 'product_view_cnt_live', 'product_click_cnt_live','item_sold_video_cnt'
                    , 'sku_order_video_cnt', 'buyer_video_cnt','viewer_video_cnt', 'unique_item_click_video_cnt', 'product_view_cnt_video',
                    'product_click_cnt_video', 'product_co_rate_live', 'product_ctr_rate_live','product_co_rate_video', 'product_ctr_rate_video']
                
                df_product = pd.DataFrame(columns=columns)
                break
        except:
            print("Connection refused by the server..")
            print("Let me sleep for 3 seconds")
            print("ZZzzzz...")
            time.sleep(3)
            print("Was a nice sleep, now let me continue...")
            continue
            
def get_product_image(x): return x['thumb_url_list'][0]
def get_product_revenue(x): return x['amount']

## 5. Data sample of live stream + video

### 5a. Live stream

In [11]:
# Get a sample live stream data of 16/07/2023
get_data_livestream(max(df_time['unix_time']))
df_livestream2 = df_livestream.copy()
df_livestream2['revenue'] = df_livestream2['revenue'].apply(revenue_clean)
df_livestream2['avg_unit_price'] = df_livestream2['avg_unit_price'].apply(avg_unit_price_clean)
df_livestream2.head(2)

,creator_id,creator_handle,creator_alias,creator_avatar,creator_type,is_commissioned,creator_bind_shop_country,live_id,start_timestamp,end_timestamp,duration,revenue,avg_unit_price,products_cnt,products_with_sales_cnt,product_views_cnt,product_clicks_cnt,items_sold_cnt,buyers_cnt,orders_created_cnt,orders_paid_cnt,avg_concurrent_users_cnt,peak_concurrent_users_cnt,co_rate,local_avg_watch_duration,local_ctr_rate,local_new_followers_cnt,local_shares_cnt,local_comments_cnt,local_likes_cnt,local_views_cnt,local_viewers_cnt
0,7225492401526703109,nt.anhhh,Ánh đâyy,{'thumb_url_list': ['https://p16-sign-sg.tikto...,3,False,,7313957381220666130,1702913411,1702919476,6065,0,0,4,0,547,16,0,0,0,0,5,17,0.0000,20,0.0138,4,0,43,868,1157,1016
1,7022862578805097499,khongcosonchannel,Quỳnh Như Khongcoson,{'thumb_url_list': ['https://p16-sign-sg.tikto...,3,False,,7313938758980717320,1702909077,1702920393,11316,358000,358000,2,1,4316,50,1,1,1,1,272,520,0.0200,52,0.0017,103,15,1750,17833,29509,22491


### 5b. Video

In [ ]:
# Get a sample live stream data of 16/07/2023
get_data_video(max(df_time['unix_time']))
df_video2 = df_video.copy()    
df_video2['creator_id'] = df_video2['creator_meta'].apply(creator_id)
df_video2['creator_handle'] = df_video2['creator_meta'].apply(creator_handle)
df_video2['creator_alias'] = df_video2['creator_meta'].apply(creator_alias)
df_video2['video_id'] = df_video2['video_meta'].apply(video_id)
df_video2['video_raw'] = df_video2['video_meta'].astype(str)
df_video2['video_name'] = df_video2['video_raw'].apply(video_name)
df_video2['video_url'] = df_video2['video_raw'].apply(video_url)
df_video2['video_publish_time'] = df_video2['video_raw'].apply(video_publish_time)
df_video2['duration'] = df_video2['video_raw'].apply(video_duration)
df_video2['revenue'] = df_video2['revenue'].apply(video_revenue)
df_video2['refunds'] = df_video2['refunds'].apply(video_refunds)
df_video2['commission_estimated'] = df_video2['commission_estimated'].apply(video_commission_estimated)
df_video2 = df_video2.drop(columns=['creator_meta', 'video_meta','product_metas','video_raw'])
df_video2 = df_video2.loc[:,['video_id', 'video_name', 'video_url', 'video_publish_time', 'duration', 'creator_id', 'creator_handle', 'creator_alias', 'view_cnt', 'like_cnt', 'comment_cnt', 'share_cnt', 'new_follower_cnt', 'item_exposure_cnt', 'item_click_cnt', 'item_sold_cnt', 'item_buyer_cnt', 'revenue', 'refunds', 'commission_estimated', 'product_refund_item_cnt', 'co_rate', 'sku_order_created_cnt', 'ctr_rate']]
df_video2.head(2)

### 5c. Order

In [ ]:
df_order.head(2)

### 5d. Order afiliate

In [ ]:
df_order_affiliate.head(2)

### 5e. Product

In [ ]:
get_data_product(max(df_time['unix_time']))
df_product['product_image'] = df_product['product_image'].apply(get_product_image)
df_product['revenue'] = df_product['revenue'].apply(get_product_revenue)
df_product['revenue_live'] = df_product['revenue_live'].apply(get_product_revenue)
df_product['revenue_video'] = df_product['revenue_video'].apply(get_product_revenue)
df_product.head(2)

## 6. Run while loop to get data of all date

### 6a. Run while loop to get data of livestream

In [38]:
def get_livestream_final():
    
    df_livestream_final = pd.DataFrame(columns = ['creator_id', 'creator_handle', 'creator_alias', 'creator_avatar'
                                   , 'creator_type', 'is_commissioned', 'creator_bind_shop_country', 'live_id'
                                   , 'start_timestamp', 'end_timestamp', 'duration', 'revenue', 'avg_unit_price'
                                   , 'products_cnt', 'products_with_sales_cnt', 'product_views_cnt'
                                   , 'product_clicks_cnt', 'items_sold_cnt', 'buyers_cnt', 'orders_created_cnt'
                                   , 'orders_paid_cnt', 'avg_concurrent_users_cnt', 'peak_concurrent_users_cnt'
                                   , 'co_rate', 'local_avg_watch_duration', 'local_ctr_rate'
                                   , 'local_new_followers_cnt', 'local_shares_cnt', 'local_comments_cnt'
                                   , 'local_likes_cnt', 'local_views_cnt', 'local_viewers_cnt'])
    
    for i in range(len(df_time)):
        x = df_time.loc[i, "date_scrape"]
        y = df_time.loc[i, "unix_time"]

        # 1. get data
        get_data_livestream(y)

        # 2. Check if scrape data has any rows
        if len(df_livestream) > 0:

            # 3. clean data
            df2 = df_livestream.copy()
            df2['revenue'] = df2['revenue'].apply(revenue_clean)
            df2['avg_unit_price'] = df2['avg_unit_price'].apply(avg_unit_price_clean)

            # 4. add column date scrape to df
            df2['date_scrape'] = x   
            df2 = df2.astype({"is_commissioned": str})
            df_livestream_final = pd.concat([df_livestream_final, df2])

            # 5. Do nothing if scrape data do not have any rows
        else:
            pass

        print('Done scrape livestream for day ' + str(x))

    file_path = "{}/Livestream/".format(file_save_path)
    file_name = "{}_tiktok_livestream_".format(brand) + str(start_date).replace('-','_') + ".xlsx"
    out_path = file_path + file_name
    writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
    df_livestream_final.set_index('creator_id', inplace=True)
    df_livestream_final = df_livestream_final.drop(columns=['creator_avatar'])  # ===> exclude some columns
    df_livestream_final.to_excel(writer, sheet_name='Sheet1')
    writer.close()
    print('Complete scraping livestream ' + '\n' +
          'File name: ' + file_name + '\n' +
          'File path: ' + out_path + '\n')
    
# get_livestream_final()    

### 6b. Run while loop to get data of video

In [39]:
def get_video_final():
    
    df_video_final = pd.DataFrame(columns = ['video_id', 'video_name', 'video_url', 'video_publish_time', 'duration'
                                             , 'creator_id', 'creator_handle', 'creator_alias', 'view_cnt', 'like_cnt'
                                             , 'comment_cnt', 'share_cnt', 'new_follower_cnt', 'item_exposure_cnt'
                                             , 'item_click_cnt', 'item_sold_cnt', 'item_buyer_cnt', 'revenue', 'refunds'
                                             , 'commission_estimated', 'product_refund_item_cnt', 'co_rate'
                                             , 'sku_order_created_cnt', 'ctr_rate'])
    
    for i in range(len(df_time)):
        x = df_time.loc[i, "date_scrape"]
        y = df_time.loc[i, "unix_time"]

        # 1. get data
        get_data_video(y)

        # 2. Check if scrape data has any rows
        if len(df_video) > 0:

            # 3. clean data
            df_video2 = df_video.copy()    
            df_video2['creator_id'] = df_video2['creator_meta'].apply(creator_id)
            df_video2['creator_handle'] = df_video2['creator_meta'].apply(creator_handle)
            df_video2['creator_alias'] = df_video2['creator_meta'].apply(creator_alias)
            df_video2['video_id'] = df_video2['video_meta'].apply(video_id)
            df_video2['video_raw'] = df_video2['video_meta'].astype(str)
            df_video2['video_name'] = df_video2['video_raw'].apply(video_name)
            df_video2['video_url'] = df_video2['video_raw'].apply(video_url)
            df_video2['video_publish_time'] = df_video2['video_raw'].apply(video_publish_time)
            df_video2['duration'] = df_video2['video_raw'].apply(video_duration)
            df_video2['revenue'] = df_video2['revenue'].apply(video_revenue)
            df_video2['refunds'] = df_video2['refunds'].apply(video_refunds)
            df_video2['commission_estimated'] = df_video2['commission_estimated'].apply(video_commission_estimated)
            df_video2 = df_video2.drop(columns=['creator_meta', 'video_meta','product_metas','video_raw'])
            df_video2 = df_video2.loc[:,['video_id', 'video_name', 'video_url', 'video_publish_time', 'duration', 'creator_id', 'creator_handle', 'creator_alias', 'view_cnt', 'like_cnt', 'comment_cnt', 'share_cnt', 'new_follower_cnt', 'item_exposure_cnt', 'item_click_cnt', 'item_sold_cnt', 'item_buyer_cnt', 'revenue', 'refunds', 'commission_estimated', 'product_refund_item_cnt', 'co_rate', 'sku_order_created_cnt', 'ctr_rate']]

            # 4. add column date scrape to df
            df_video2['date_scrape'] = x   
            df_video_final = pd.concat([df_video_final, df_video2])

            # 5. Do nothing if scrape data do not have any rows
        else:
            pass

        print('Done scrape video for day ' + str(x))

    file_path = "{}/Video/".format(file_save_path)
    file_name = "{}_tiktok_video_".format(brand) + str(start_date).replace('-','_') + ".xlsx"
    out_path = file_path + file_name
    writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
    df_video_final.set_index('video_id', inplace=True)
    df_video_final.to_excel(writer, sheet_name='Sheet1')
    writer.close()
    print('Complete scraping Video ' + '\n' +
          'File name: ' + file_name + '\n' +
          'File path: ' + out_path + '\n')
    
# get_video_final()  

### 6c. Parse Order data to excel folder

In [40]:
def get_order_final():
    file_path = "{}/Orders/".format(file_save_path)
    file_name = "{}_tiktok_orders_".format(brand) + str(start_date).replace('-','_') + ".xlsx"
    out_path = file_path + file_name
    writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
    df_order1 = df_order.copy()
    df_order1['Order ID'] = df_order1['Order ID'].astype(str)
    df_order1['SKU ID'] = df_order1['SKU ID'].astype(str)
    df_order1.set_index('Order ID', inplace=True)
    df_order1.to_excel(writer, sheet_name='Sheet1')
    writer.close()
    print('Complete scraping orders ' + '\n' +
          'File name: ' + file_name + '\n' +
          'File path: ' + out_path + '\n')
    
# get_order_final()    

### 6d. Run while loop to get data of affiliate

In [41]:
def get_order_affiliate_final():
    file_path = "{}/Affiliate Orders/".format(file_save_path)
    file_name = "{}_tiktok_affiliate_orders_".format(brand) + str(start_date).replace('-','_') + ".xlsx"
    out_path = file_path + file_name
    writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
    df_order_affiliate.set_index('Order ID', inplace=True)
    df_order_affiliate.to_excel(writer, sheet_name='Sheet1')
    writer.close()
    print('Complete scraping affiliate orders ' + '\n' +
          'File name: ' + file_name + '\n' +
          'File path: ' + out_path + '\n')
    
# get_order_affiliate_final()    

### 6e. Run while loop to get data of product

In [42]:
def get_product_final():
    
    df_product_final = pd.DataFrame(columns = ['shop_id', 'product_id', 'product_name', 'product_image', 'revenue', 'revenue_live'
                                               , 'revenue_video', 'buyer_cnt', 'item_sold_cnt', 'sku_order_cnt', 'item_sold_live_cnt'
                                               , 'buyer_live_cnt', 'viewer_live_cnt', 'unique_item_click_live_cnt'
                                               , 'product_view_cnt_live', 'product_click_cnt_live', 'item_sold_video_cnt'
                                               , 'sku_order_video_cnt', 'buyer_video_cnt', 'viewer_video_cnt'
                                               , 'unique_item_click_video_cnt', 'product_view_cnt_video', 'product_click_cnt_video'
                                               , 'product_co_rate_live', 'product_ctr_rate_live', 'product_co_rate_video'
                                               , 'product_ctr_rate_video'])
    
    for i in range(len(df_time)):
        x = df_time.loc[i, "date_scrape"]
        y = df_time.loc[i, "unix_time"]

        # 1. get data
        get_data_product(y)

        # 2. Check if scrape data has any rows
        if len(df_product) > 0:

            # 3. clean data
            df_product2 = df_product.copy()    
            df_product2['product_image'] = df_product['product_image'].apply(get_product_image)
            df_product2['revenue'] = df_product2['revenue'].apply(get_product_revenue)
            df_product2['revenue_live'] = df_product2['revenue_live'].apply(get_product_revenue)
            df_product2['revenue_video'] = df_product2['revenue_video'].apply(get_product_revenue)

            # 4. add column date scrape to df
            df_product2['date_scrape'] = x   
            df_product_final = pd.concat([df_product_final, df_product2])

            # 5. Do nothing if scrape data do not have any rows
        else:
            pass

        print('Done scrape product for day ' + str(x))

    file_path = "{}/Product/".format(file_save_path)
    file_name = "{}_tiktok_product_".format(brand) + str(start_date).replace('-','_') + ".xlsx"
    out_path = file_path + file_name
    writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
    df_product_final.set_index('product_id', inplace=True)
    df_product_final.to_excel(writer, sheet_name='Sheet1')
    writer.close()
    print('Complete scraping Product ' + '\n' +
          'File name: ' + file_name + '\n' +
          'File path: ' + out_path)
    
# get_product_final()  

In [43]:
if '1' in domain:
    print('1. Start scraping livestream: ')
    get_livestream_final()  
if '2' in domain:
    print('2. Start scraping video: ')
    get_video_final()  
if '3' in domain:     
    print('3. Start scraping Orders: ')
    get_order_final()
if '4' in domain:     
    print('4. Start scraping Affiliate orders: ')
    get_order_affiliate_final()
if '5' in domain:     
    print('5. Start scraping Product: ')
    get_product_final()    

1. Start scraping livestream: 
Done scrape livestream for day 2023-12-01
Done scrape livestream for day 2023-12-02
Done scrape livestream for day 2023-12-03
Done scrape livestream for day 2023-12-04
Done scrape livestream for day 2023-12-05
Done scrape livestream for day 2023-12-06
Done scrape livestream for day 2023-12-07
Done scrape livestream for day 2023-12-08
Done scrape livestream for day 2023-12-09
Done scrape livestream for day 2023-12-10
Done scrape livestream for day 2023-12-11
Done scrape livestream for day 2023-12-12
Done scrape livestream for day 2023-12-13
Done scrape livestream for day 2023-12-14
Done scrape livestream for day 2023-12-15
Done scrape livestream for day 2023-12-16
Done scrape livestream for day 2023-12-17
Complete scraping livestream 
File name: cerave_tiktok_livestream_2023_12_01.xlsx
File path: C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/Test/Livestream/cerave_tiktok_livestream_2023_12_01.xlsx

2. Start scr